## Example of approximations to the Poisson problem with different boundary conditions


The problem to solve is: 


\begin{align}
-\Delta u &= f \;\;\;\;\; \text{in } \Omega \\
u &= g \;\;\;\;\; \text{in } \partial \Omega_{int} \\
\hat{n} \cdot \nabla u &= h \;\;\;\;\; \text{in } \partial \Omega_{ext} \\
\end{align}

For that we impose the weak version of it:

Find $u$ in $V = H^1(\Omega, f) \cap H^1_0(\Omega, \partial \Omega_{int})$ (that is with elements vanishing at $\partial \Omega_{int}$) such that:,

$$
\int_{\Omega} \nabla v \cdot \nabla u \; d\Omega 
- \int_{\Omega} v \; f \; d\Omega 
- \oint_{\partial \Omega_{ext}} v \; h \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in V
$$

If we obtain a solution $u$ and it is sufficiently smooth, we can integrate by parts the first term and obtain: 

$$
\int_{\Omega}  v \; (-\Delta u - f) \; d\Omega 
+ \oint_{\partial \Omega_{ext}} v \; (\hat{n} \cdot \nabla u - h) \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in V
$$

Taking $v$ arbitrary but of compact support inside $Ω$ we see that $u$ must satisfy:

$$
-\Delta u = f \;\;\;\;\; \text{in } \Omega,
$$
taking now $v$ arbitrary but different from zero in a neighborhood of the exterior boundary, we see that it must satisfy also the Neumann condition:

$$
\hat{n} \cdot \nabla u = h \;\;\;\;\; \text{in } \partial \Omega_{ext}.
$$

The Dirichlet condition is automatic for this choice of functional space.




To solve the problem we shall use the a Julia package `Gridap.jl`. This example is a mixture of several examples in the Gridap tutorial.

In [ ]:
using Gridap
using GridapGmsh
#import Gmsh: gmsh # NEVER use #using Gmsh
#mkdir("models")
#mkdir("images")

In [ ]:
plot_s = true
if plot_s
    using GridapMakie, GLMakie #Para graficar 
    using FileIO #Gráficos y salidas
    GLMakie.activate!(inline=true) # For windows on the notebook itself. Comment out if you want the as pop-out plots.
end

We will use a mesh previously built with the `gmsh` library. In the `models` directory you will find a *script* with the `.geo` extension (`rectangle_hole.geo`), which was used to build the example. Based on this script, and following the `gmsh` tutorial, you can build different meshes. Other libraries can also be used to build meshes. These are imported into the **Gridap** infrastructure, and with them the triangulation to be used is constructed. Note that in the script, the two boundaries are named: the external (rectangular) one as `"ext"` and the internal (circle) one as `"int"`.

In [ ]:
#model = GmshDiscreteModel("models/rectangle_hole_fine.msh")
model = GmshDiscreteModel("models/rectangle_hole_coarse.msh")
#model = GmshDiscreteModel("models/rectangle_hole_finer.msh")


In [ ]:
Ω = Triangulation(model)

In [ ]:
degree = 2
dΩ = Measure(Ω,degree)

In [ ]:
if plot_s
    fig, ax = plot(Ω)
    ax.aspect = AxisAspect(2)
    wireframe!(Ω, color=:black, linewidth=1)
    scatter!(Ω, marker=:star8, markersize=4, color=:blue)
    fig
end

In [ ]:
full_dirichlet = false
int_dirichlet = false
#full_dirichlet = true
int_dirichlet = true

Once we have the mesh, we begin to define the finite elements that we will use. In this case, we will use Lagrangian elements of **order 1** that will satisfy a Dirichlet condition on the region $\partial \Omega_{int}$. When the mesh is built, this region has been marked as the interior boundary of the rectangle with the `tag` `"int"`.

In [ ]:
order = 2
reffe = ReferenceFE(lagrangian,Float64,order)
if int_dirichlet
    dirichlet_tags="int, " 
elseif full_dirichlet
    dirichlet_tags=["int","ext"]
end
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags = dirichlet_tags)


In this example, we are going to test our code using a *solution* given by the function $u_e(x,y) =  4((x-x_0)^2 - (y-y_0)^3) - 5y$.

Thus, $\Delta u_e = -f = 8 - 24(y-y0)$ and the Neumann condition will be given by $h = \hat{n} \cdot \nabla u_e$.

In [ ]:
x0 = 0.5
y0 = 0.5
ue(x) = 4*((x[1]-x0)^2 - (x[2]-y0)^3) - 5.0*x[2] # "exact solution"

if plot_s
    fig, ax, plt = plot(Ω, ue, shading=false)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end


In [ ]:
# internal Dirichlet boundary condition
U = TrialFESpace(V,ue)
Γ₁ = BoundaryTriangulation(model,tags=dirichlet_tags)
n₁ = get_normal_vector(Γ₁)

if plot_s
    fig, ax , plt  = plot(Γ₁,ue, colormap=:heat, linewidth=10)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

In [ ]:
if int_dirichlet
    neumann_tags = "ext"
    #neumanntags = "int"
    Γ = BoundaryTriangulation(model,tags=neumann_tags)
    dΓ = Measure(Γ,degree)
    nb = get_normal_vector(Γ)
end


To make sure every thing is OK, we plot the values of $\hat{n}\cdot \nabla u_e$ at the external boundary.

In [ ]:
if int_dirichlet && plot_s
    fig, ax , plt = plot(Γ, (nb ⋅ ∇(ue)), colormap=:algae, linewidth=10)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

We now define the weak problem in abstract form: 

In [ ]:
f(x) = -8.0 + 24*(x[2] - y0) # source
#h(x) =  #external Neumann bc.
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ  #  in a(u,v) goes all the terms linear in the unknown, u. 
if full_dirichlet
    b(v) = ∫(v*f )*dΩ # 
elseif int_dirichlet
    b(v) = ∫(v*f )*dΩ + ∫(v*(nb ⋅ ∇(ue)))*dΓ #  here everything else.
end

From here the package  **Gridap.jl** generates a equation system of the form $Ax=b$ and solves it for the finite element version of $u$.

In [ ]:
op = AffineFEOperator(a,b,U,V)


In [ ]:
ls = LUSolver()
solver = LinearFESolver(ls)

In [ ]:
uh = solve(solver,op)


In [ ]:
if plot_s
    fig, ax, plt = plot(Ω, uh, shading=NoShading)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end


In [ ]:
if full_dirichlet
    writevtk(Ω,"images/solución_dir",cellfields=["uh_dir" => uh])
elseif int_dirichlet
    writevtk(Ω,"images/solución_newmann",cellfields=["uh_neu" => uh])
end

We now are going to validate our approximation by comparing it with the exact solution. For that we introduce several tools:

In [ ]:
e = ue - uh

In [ ]:
if plot_s
    fig, ax, plt = plot(Ω, e
        , shading=NoShading
        )
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end
fig

In [ ]:
if full_dirichlet
    writevtk(Ω,"images/error_dir",cellfields=["e_dir" => e])
elseif int_dirichlet
    writevtk(Ω,"images/error_newmann",cellfields=["e_neu" => e])
end

Next we compute the $L^2$ and $H^1$ norm for the error. 

In [ ]:
el2 = sqrt(sum( ∫( e*e )*dΩ ))
println("l2 error = ",el2)
eh1 = sqrt(sum( ∫( e*e + ∇(e)⋅∇(e) )*dΩ ))
uh1 = sqrt(sum( ∫( uh*uh + ∇(uh)⋅∇(uh) )*dΩ ))
println("h1 error = ",eh1/uh1)

In [ ]:
if plot_s
    fig, ax, plt = plot(Ω, ∇(e)⋅∇(e), shading=NoShading)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end